<a href="https://colab.research.google.com/github/handeo/SBTi-finance-tool/blob/main/examples/2_quick_temp_score_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SBTi-Finance Tool - Quick Temperature Score Calculation
This notebook provides a simple example of the SBTi-Finance Tool. It shows how to use it to calculate the temperature score for companies, aggregate them to a portfolio level to get the portfolio temperature score. It also shows you how to calculate the portfolio coverage.

Please see the [methodology](https://sciencebasedtargets.org/wp-content/uploads/2020/09/Temperature-Rating-Methodology-V1.pdf), [guidance](https://sciencebasedtargets.org/wp-content/uploads/2020/10/Financial-Sector-Science-Based-Targets-Guidance-Pilot-Version.pdf) and the [technical documentation](https://sciencebasedtargets.github.io/SBTi-finance-tool/) for more details.

See 1_analysis_example (on [Colab](https://colab.research.google.com/github/OFBDABV/SBTi/blob/master/examples/1_analysis_example.ipynb) or [Github](https://github.com/ScienceBasedTargets/SBTi-finance-tool/blob/master/examples/1_analysis_example.ipynb)) for more in depth example of how to work with Jupyter Notebooks in general and SBTi notebooks in particular.

### Install the SBTi Python module
This is only required if you have not already installed the module.

In [1]:
%pip install sbti-finance-tool

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 37.7 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.6.1
    Uninstalling pydantic-2.6.1:
      Successfully uninstalled pydantic-2.6.1


In [2]:
import SBTi
from SBTi.data.excel import ExcelProvider
from SBTi.portfolio_aggregation import PortfolioAggregationMethod
from SBTi.portfolio_coverage_tvp import PortfolioCoverageTVP
from SBTi.temperature_score import TemperatureScore, Scenario
from SBTi.target_validation import TargetProtocol
from SBTi.interfaces import ETimeFrames, EScope
import pandas as pd

## Download the dummy data provider
We have prepared dummy data for you to be able to run the tool as it is to familiarise yourself with how it works. To use your own data; please check out to the [Data Requirements section](https://sciencebasedtargets.github.io/SBTi/DataRequirements.html) of the technical documentation for more details on data requirements and formatting.

*The dummy data may include some company names, but the data associated with those company names is completely random and any similarities with real world data is purely coincidental.


In [39]:
import urllib.request
import os

if not os.path.isdir("data"):
    os.mkdir("data")
if not os.path.isfile("data/Data_Provider_norevenue2.xlsx"):
    urllib.request.urlretrieve("https://github.com/handeo/SBTi-finance-tool/raw/main/data/Data_Provider_norevenue2.xlsx", "data/Data_Provider_norevenue2.xlsx")
if not os.path.isfile("data/portfolio_data2.csv"):
    urllib.request.urlretrieve("https://github.com/handeo/SBTi-finance-tool/raw/main/data/portfolio_data2.csv", "data/portfolio_data2.csv")

##### Logging
The SBTi module uses the Python standard library logging utilities to send log messages. The log level can be changed according to the user's needs.

In [40]:
import logging
root_logger = logging.getLogger()
root_logger.setLevel("INFO")

## Create a data provider
Data providers let you connect to the data source of your choice. In this case we are connecting to Excel as a data provider. For all available dataproviders check the implementation [here](https://github.com/ScienceBasedTargets/SBTi-finance-tool/tree/master/SBTi/data)

In [41]:
provider = ExcelProvider(path="data/Data_Provider_norevenue2.xlsx")

## Load your portfolio
In our case the portfolio is stored as a CSV file. The portfolio should at least have an "id" (the identifier of the company) and a "proportion" (the weight of the company in your portfolio e.g. the value of the shares you hold) column.

Please see the technical documentation on [Data Legends](https://sciencebasedtargets.github.io/SBTi/Legends.html#) for details on data requirements.

In [42]:
df_portfolio = pd.read_csv("data/portfolio_data2.csv", encoding="iso-8859-1")

In [43]:
df_portfolio.head(5)

,ï»¿company_name,company_id,company_isin,company_lei,weights,investment_value,engagement_target
0,Company 6,Customer6,NaN,NaN,1.65%,3101504,NaN
1,Company 116,Customer116,NaN,NaN,15.95%,30037648,NaN
2,Company 117,Customer117,NaN,NaN,11.47%,21590662,NaN
3,Company 119,Customer119,NaN,789000USXJ5EA7GBM139,7.81%,14708835,NaN
4,Company 121,Customer121,NaN,789000SVZUQTHE1Q7X60,8.61%,16208861,NaN


In [44]:
df_portfolio.columns = ['company_name'] + list(df_portfolio.columns[1:])

In [45]:
df_portfolio.head(5)

,company_name,company_id,company_isin,company_lei,weights,investment_value,engagement_target
0,Company 6,Customer6,NaN,NaN,1.65%,3101504,NaN
1,Company 116,Customer116,NaN,NaN,15.95%,30037648,NaN
2,Company 117,Customer117,NaN,NaN,11.47%,21590662,NaN
3,Company 119,Customer119,NaN,789000USXJ5EA7GBM139,7.81%,14708835,NaN
4,Company 121,Customer121,NaN,789000SVZUQTHE1Q7X60,8.61%,16208861,NaN


To load the data from the data provider, we have to pass a list of IPortfolioCompany instances. The module has a strict [data model](https://sciencebasedtargets.github.io/SBTi-finance-tool/autoapi/SBTi/interfaces/index.html) to convert Pandas Dataframe to the right object types we supplied a utility function.


In [46]:
companies = SBTi.utils.dataframe_to_portfolio(df_portfolio)

## Calculate the temperature scores
In the amended portfolio you'll find your original portfolio, amended with the emissions and the temperature score.

In [52]:
temperature_score = TemperatureScore(                  # all available options:
    time_frames=list(SBTi.interfaces.ETimeFrames),     # ETimeFrames: SHORT MID and LONG
    scopes=[EScope.S1S2, EScope.S3, EScope.S1S2S3],    # EScopes: S3, S1S2 and S1S2S3
    aggregation_method=PortfolioAggregationMethod.WATS # Options for the aggregation method are WATS, TETS, AOTS, MOTS, EOTS, ECOTS, and ROTS.
)
amended_portfolio = temperature_score.calculate(data_providers=[provider], portfolio=companies)

Status code from fetching the CTA file: 200, 200 = OK


For every company the tool assigns a score for all the requested timeframe and scope combinations. In this example we used the full set resulting in 9 scores per company as displayed below:

In [53]:
amended_portfolio[['company_name', 'time_frame', 'scope', 'temperature_score']].head(9)

,company_name,time_frame,scope,temperature_score
0,Customer X,SHORT,S1S2,3.20
1,Customer X,SHORT,S3,3.20
2,Customer X,MID,S1S2,0.94
3,Customer X,MID,S3,3.20
4,Customer X,LONG,S1S2,3.20
5,Customer X,LONG,S3,3.20
6,Customer Y,SHORT,S1S2,3.20
7,Customer Y,SHORT,S3,3.20
8,Customer Y,MID,S1S2,0.94


## Calculate the aggregated temperature score
Calculate an aggregated temperature score. This can be done using different aggregation methods. Here we'll use the "Weighted Average Temperature Score" (WATS) by initializing the TemperatureScore Object with PortfolioAggregationMethod.WATS. For more details, please refer to notebook 4 (on [Colab](https://colab.research.google.com/github/OFBDABV/SBTi/blob/master/examples/4_portfolio_aggregations.ipynb) or [GitHub](https://github.com/ScienceBasedTargets/SBTi-finance-tool/blob/master/examples/4_portfolio_aggregations.ipynb)) and the [methodology document](https://sciencebasedtargets.org/wp-content/uploads/2020/09/Temperature-Rating-Methodology-V1.pdf) sections 3.2. The temperature scores are calculated per time-frame/scope combination.


In [54]:
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)

In [50]:
# Here we cast a ScoreAggregation object to a Pandas Dataframe for viewing the temp scores in a human readable way
pd.DataFrame(aggregated_scores.dict()).applymap(lambda x: round(x['all']['score'], 2))

,short,mid,long
S1S2,3.2,0.94,3.2
S3,3.2,3.20,3.2
S1S2S3,3.2,3.20,3.2


## Portfolio coverage

The portfolio coverage provides insights in the proportion of the portfolio that has set SBTi-approved GHG emissions reduction targets. Only companies with SBTi-status "Approved" are included in the portfolio coverage.

To calculate the portfolio coverage we use the same aggregation methods we use for the Portfolio Temperature Score. In this example we use the "Weighted Average Temperature Score" (WATS). For more details on aggregation methods and the portfolio coverage method, please refer to the [methodology document](https://sciencebasedtargets.org/wp-content/uploads/2020/09/Temperature-Rating-Methodology-V1.pdf) sections 3.2 and also turn to notebook 4 (on [Colab](https://colab.research.google.com/github/OFBDABV/SBTi/blob/master/examples/4_portfolio_aggregations.ipynb) or [GitHub](https://github.com/ScienceBasedTargets/SBTi-finance-tool/blob/master/examples/4_portfolio_aggregations.ipynb)) for more aggregation examples.

In [51]:
portfolio_coverage_tvp = PortfolioCoverageTVP()
coverage = portfolio_coverage_tvp.get_portfolio_coverage(amended_portfolio.copy(), PortfolioAggregationMethod.WATS)
print("Portfolio coverage is: {c:.2f}%".format(c=coverage))

Portfolio coverage is: 0.00%
